# Module 5: Portfolio Optimization and Risk Management

**CRITICAL FOR QUANTITATIVE FINANCE CAREERS**

This module covers Modern Portfolio Theory, Markowitz optimization, risk parity, and practical portfolio construction.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from scipy.optimize import minimize
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Portfolio Optimization Module Loaded!")

## 1. Modern Portfolio Theory (MPT)

**Markowitz Mean-Variance Optimization:**

Minimize portfolio variance:
$$\min_w \frac{1}{2} w^T \Sigma w$$

Subject to:
- $w^T \mu = \mu_p$ (target return)
- $w^T \mathbf{1} = 1$ (fully invested)
- $w_i \geq 0$ (no short selling, optional)

**Key Concepts:**
- **Efficient Frontier**: Set of optimal portfolios
- **Sharpe Ratio**: $(r_p - r_f) / \sigma_p$
- **Diversification**: Reduces idiosyncratic risk

In [ ]:
# Download data for diverse portfolio
tickers = ['SPY', 'TLT', 'GLD', 'VNQ', 'EFA', 'EEM']  # Stocks, Bonds, Gold, REITs, Developed, Emerging
start_date = '2015-01-01'
end_date = '2024-11-01'

print("Downloading data for portfolio assets...")
data = yf.download(tickers, start=start_date, end=end_date, progress=False)['Close']
returns = data.pct_change().dropna()

print(f"\nAssets: {tickers}")
print(f"SPY: S&P 500 ETF")
print(f"TLT: 20+ Year Treasury Bonds")
print(f"GLD: Gold")
print(f"VNQ: Real Estate")
print(f"EFA: Developed Markets (ex-US)")
print(f"EEM: Emerging Markets")

print(f"\nData range: {returns.index[0]} to {returns.index[-1]}")
print(f"Observations: {len(returns)}")

# Calculate summary statistics
ann_returns = returns.mean() * 252
ann_vol = returns.std() * np.sqrt(252)
sharpe = ann_returns / ann_vol

summary_df = pd.DataFrame({
    'Ann. Return': ann_returns,
    'Ann. Volatility': ann_vol,
    'Sharpe Ratio': sharpe
})

print("\nAsset Statistics (Annualized):")
print(summary_df.round(4))

In [ ]:
# Calculate correlation matrix
corr_matrix = returns.corr()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Correlation heatmap
ax = axes[0]
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
           square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Correlation Matrix', fontsize=14, fontweight='bold')

# Risk-Return scatter
ax = axes[1]
ax.scatter(summary_df['Ann. Volatility'], summary_df['Ann. Return'], s=200, alpha=0.6)
for ticker in tickers:
    ax.annotate(ticker, 
               (summary_df.loc[ticker, 'Ann. Volatility'], 
                summary_df.loc[ticker, 'Ann. Return']),
               fontsize=12, fontweight='bold')
ax.set_xlabel('Annualized Volatility', fontsize=12)
ax.set_ylabel('Annualized Return', fontsize=12)
ax.set_title('Risk-Return Profile', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Portfolio Optimization Functions

In [ ]:
class PortfolioOptimizer:
    """Portfolio optimization with various objectives."""
    
    def __init__(self, returns):
        """
        Initialize optimizer.
        
        Parameters:
        -----------
        returns : pd.DataFrame
            Asset returns (T x N)
        """
        self.returns = returns
        self.mean_returns = returns.mean() * 252  # Annualized
        self.cov_matrix = returns.cov() * 252     # Annualized
        self.n_assets = len(returns.columns)
    
    def portfolio_stats(self, weights):
        """
        Calculate portfolio return, volatility, and Sharpe ratio.
        
        Returns:
        --------
        tuple : (return, volatility, sharpe_ratio)
        """
        portfolio_return = np.dot(weights, self.mean_returns)
        portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(self.cov_matrix, weights)))
        sharpe_ratio = portfolio_return / portfolio_vol
        return portfolio_return, portfolio_vol, sharpe_ratio
    
    def min_variance_portfolio(self, long_only=True):
        """
        Find minimum variance portfolio.
        
        Parameters:
        -----------
        long_only : bool
            If True, no short selling allowed
        
        Returns:
        --------
        dict : Optimal weights and statistics
        """
        def portfolio_variance(weights):
            return np.dot(weights.T, np.dot(self.cov_matrix, weights))
        
        # Constraints
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        
        # Bounds
        bounds = tuple((0, 1) if long_only else (-1, 1) for _ in range(self.n_assets))
        
        # Initial guess
        x0 = np.array([1/self.n_assets] * self.n_assets)
        
        # Optimize
        result = minimize(portfolio_variance, x0, method='SLSQP', 
                        bounds=bounds, constraints=constraints)
        
        weights = result.x
        ret, vol, sharpe = self.portfolio_stats(weights)
        
        return {
            'weights': weights,
            'return': ret,
            'volatility': vol,
            'sharpe': sharpe
        }
    
    def max_sharpe_portfolio(self, long_only=True, risk_free_rate=0.02):
        """
        Find maximum Sharpe ratio portfolio (tangency portfolio).
        
        Parameters:
        -----------
        long_only : bool
            If True, no short selling allowed
        risk_free_rate : float
            Annual risk-free rate
        
        Returns:
        --------
        dict : Optimal weights and statistics
        """
        def neg_sharpe(weights):
            ret, vol, _ = self.portfolio_stats(weights)
            return -(ret - risk_free_rate) / vol
        
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = tuple((0, 1) if long_only else (-1, 1) for _ in range(self.n_assets))
        x0 = np.array([1/self.n_assets] * self.n_assets)
        
        result = minimize(neg_sharpe, x0, method='SLSQP', 
                        bounds=bounds, constraints=constraints)
        
        weights = result.x
        ret, vol, sharpe = self.portfolio_stats(weights)
        
        return {
            'weights': weights,
            'return': ret,
            'volatility': vol,
            'sharpe': sharpe
        }
    
    def risk_parity_portfolio(self):
        """
        Find risk parity portfolio (equal risk contribution).
        
        Returns:
        --------
        dict : Optimal weights and statistics
        """
        def risk_contribution_objective(weights):
            """Minimize difference in risk contributions."""
            portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(self.cov_matrix, weights)))
            marginal_contrib = np.dot(self.cov_matrix, weights)
            risk_contrib = weights * marginal_contrib / portfolio_vol
            target_risk = portfolio_vol / self.n_assets
            return np.sum((risk_contrib - target_risk)**2)
        
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
        bounds = tuple((0, 1) for _ in range(self.n_assets))
        x0 = np.array([1/self.n_assets] * self.n_assets)
        
        result = minimize(risk_contribution_objective, x0, method='SLSQP',
                        bounds=bounds, constraints=constraints)
        
        weights = result.x
        ret, vol, sharpe = self.portfolio_stats(weights)
        
        return {
            'weights': weights,
            'return': ret,
            'volatility': vol,
            'sharpe': sharpe
        }
    
    def efficient_frontier(self, n_portfolios=50, long_only=True):
        """
        Generate efficient frontier.
        
        Parameters:
        -----------
        n_portfolios : int
            Number of portfolios to generate
        long_only : bool
            If True, no short selling
        
        Returns:
        --------
        pd.DataFrame : Efficient frontier portfolios
        """
        # Get min and max returns
        min_var_port = self.min_variance_portfolio(long_only)
        max_ret = self.mean_returns.max()
        min_ret = min_var_port['return']
        
        target_returns = np.linspace(min_ret, max_ret, n_portfolios)
        
        frontier_results = []
        
        for target_ret in target_returns:
            # Minimize variance for target return
            def portfolio_variance(weights):
                return np.dot(weights.T, np.dot(self.cov_matrix, weights))
            
            constraints = [
                {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
                {'type': 'eq', 'fun': lambda w: np.dot(w, self.mean_returns) - target_ret}
            ]
            
            bounds = tuple((0, 1) if long_only else (-1, 1) for _ in range(self.n_assets))
            x0 = np.array([1/self.n_assets] * self.n_assets)
            
            try:
                result = minimize(portfolio_variance, x0, method='SLSQP',
                                bounds=bounds, constraints=constraints)
                
                if result.success:
                    weights = result.x
                    ret, vol, sharpe = self.portfolio_stats(weights)
                    frontier_results.append({
                        'return': ret,
                        'volatility': vol,
                        'sharpe': sharpe
                    })
            except:
                pass
        
        return pd.DataFrame(frontier_results)

# Initialize optimizer
optimizer = PortfolioOptimizer(returns)
print("\nPortfolio Optimizer initialized!")

## 3. Optimal Portfolios

In [ ]:
# Calculate optimal portfolios
print("Calculating optimal portfolios...\n")

equal_weight = {
    'weights': np.array([1/len(tickers)] * len(tickers)),
    **optimizer.portfolio_stats(np.array([1/len(tickers)] * len(tickers)))
}
equal_weight['return'], equal_weight['volatility'], equal_weight['sharpe'] = \
    optimizer.portfolio_stats(equal_weight['weights'])

min_var = optimizer.min_variance_portfolio()
max_sharpe = optimizer.max_sharpe_portfolio()
risk_parity = optimizer.risk_parity_portfolio()

# Create comparison table
portfolios = {
    'Equal Weight': equal_weight,
    'Min Variance': min_var,
    'Max Sharpe': max_sharpe,
    'Risk Parity': risk_parity
}

# Display weights
weights_df = pd.DataFrame({name: port['weights'] for name, port in portfolios.items()},
                         index=tickers)

print("PORTFOLIO ALLOCATIONS")
print("="*80)
print(weights_df.round(4))

# Display statistics
stats_df = pd.DataFrame({
    'Return': [port['return'] for port in portfolios.values()],
    'Volatility': [port['volatility'] for port in portfolios.values()],
    'Sharpe': [port['sharpe'] for port in portfolios.values()]
}, index=portfolios.keys())

print("\nPORTFOLIO STATISTICS (Annualized)")
print("="*80)
print(stats_df.round(4))

In [ ]:
# Visualize allocations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for idx, (name, port) in enumerate(portfolios.items()):
    ax = axes[idx]
    weights_series = pd.Series(port['weights'], index=tickers)
    weights_series[weights_series > 0.01].plot(kind='pie', ax=ax, autopct='%1.1f%%')
    ax.set_title(f"{name}\nReturn: {port['return']:.2%}, Vol: {port['volatility']:.2%}, Sharpe: {port['sharpe']:.2f}",
                fontsize=12, fontweight='bold')
    ax.set_ylabel('')

plt.tight_layout()
plt.show()

## 4. Efficient Frontier

In [ ]:
# Generate efficient frontier
print("Generating efficient frontier...")
frontier = optimizer.efficient_frontier(n_portfolios=100)

# Plot efficient frontier
fig, ax = plt.subplots(figsize=(14, 8))

# Efficient frontier
ax.plot(frontier['volatility'], frontier['return'], 'b-', linewidth=3, label='Efficient Frontier')

# Individual assets
ax.scatter(summary_df['Ann. Volatility'], summary_df['Ann. Return'], 
          s=100, alpha=0.6, c='gray', label='Individual Assets')
for ticker in tickers:
    ax.annotate(ticker, 
               (summary_df.loc[ticker, 'Ann. Volatility'], 
                summary_df.loc[ticker, 'Ann. Return']),
               fontsize=10, alpha=0.7)

# Optimal portfolios
colors = ['green', 'red', 'purple', 'orange']
for (name, port), color in zip(portfolios.items(), colors):
    ax.scatter(port['volatility'], port['return'], s=200, c=color, marker='*', 
              edgecolors='black', linewidths=2, label=name, zorder=5)

ax.set_xlabel('Annualized Volatility', fontsize=14)
ax.set_ylabel('Annualized Return', fontsize=14)
ax.set_title('Efficient Frontier and Optimal Portfolios', fontsize=16, fontweight='bold')
ax.legend(loc='best', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nEfficient Frontier generated successfully!")

## 5. Backtesting Portfolios

In [ ]:
# Calculate portfolio returns over time
portfolio_returns = {}

for name, port in portfolios.items():
    portfolio_returns[name] = (returns * port['weights']).sum(axis=1)

portfolio_returns_df = pd.DataFrame(portfolio_returns)

# Calculate cumulative returns
cumulative_returns = (1 + portfolio_returns_df).cumprod()

# Plot cumulative performance
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# Cumulative returns
ax = axes[0]
cumulative_returns.plot(ax=ax, linewidth=2)
ax.set_title('Cumulative Returns: Portfolio Strategies', fontsize=14, fontweight='bold')
ax.set_ylabel('Cumulative Return', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

# Rolling Sharpe ratio (252-day window)
ax = axes[1]
rolling_sharpe = portfolio_returns_df.rolling(252).mean() / portfolio_returns_df.rolling(252).std() * np.sqrt(252)
rolling_sharpe.plot(ax=ax, linewidth=2)
ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax.set_title('Rolling Sharpe Ratio (1-Year Window)', fontsize=14, fontweight='bold')
ax.set_ylabel('Sharpe Ratio', fontsize=12)
ax.set_xlabel('Date', fontsize=12)
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Final statistics
final_values = cumulative_returns.iloc[-1]
total_returns = (final_values - 1) * 100

print("\nBACKTEST RESULTS")
print("="*80)
print(f"Period: {returns.index[0]} to {returns.index[-1]}")
print("\nTotal Returns:")
for name in portfolios.keys():
    print(f"  {name:.<20} {total_returns[name]:>8.2f}%")

print(f"\nBest Performing: {total_returns.idxmax()} ({total_returns.max():.2f}%)")

## Key Takeaways

### Portfolio Theory
1. **Diversification**: Reduces risk without sacrificing return
2. **Efficient Frontier**: Optimal portfolios for each risk level
3. **Sharpe Ratio**: Risk-adjusted performance metric
4. **Correlation**: Low correlation assets improve diversification

### Optimization Strategies
1. **Minimum Variance**: Lowest risk, conservative
2. **Maximum Sharpe**: Best risk-adjusted returns
3. **Risk Parity**: Equal risk contribution from each asset
4. **Equal Weight**: Simple, works well empirically

### Practical Considerations
- **Estimation Error**: Sample statistics ≠ true parameters
- **Rebalancing**: Transaction costs and turnover
- **Constraints**: Regulatory, liquidity, practical limits
- **Robustness**: Use regularization, robust estimators

**CRITICAL FOR CAREERS:**
- Portfolio construction is core PM/analyst skill
- Understanding MPT essential for interviews
- Real-world implementations more complex
- Factor models, Black-Litterman often preferred